### Numpy code

In [2]:
import numpy as np

# Example: Large matrices (adjust size as needed)
n = 7000  # For very large matrices, ensure you have enough RAM
A = np.random.rand(n, n).astype(np.float32)
B = np.random.rand(n, n).astype(np.float32)

C = np.dot(A, B)  # warm-up and Matrix multiplication

%timeit -r 2 -o np.dot(A, B)

print(f"Result shape: {C.shape}")
print(f"Result type: {C.dtype}")

1.05 s ± 748 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)
Result shape: (7000, 7000)
Result type: float32


Celda de CuPy (GPU)

In [4]:
import cupy as cp

# Mover las matrices A y B a la memoria de la GPU
A_gpu = cp.asarray(A)
B_gpu = cp.asarray(B)

# Warm-up (importante para ignorar el tiempo de inicialización de CUDA)
C_gpu = cp.dot(A_gpu, B_gpu)

# Medir el tiempo en GPU
# Usamos cp.cuda.Stream.null.synchronize() para asegurar que la GPU termine antes de parar el cronómetro
%timeit -r 2 -o cp.dot(A_gpu, B_gpu); cp.cuda.Stream.null.synchronize()

156 ms ± 63 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


<TimeitResult : 156 ms ± 63 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)>

### Análisis de Resultados: Multiplicación de Matrices ($7000 \times 7000$)

Al comparar los tiempos obtenidos, se observa una mejora drástica en el rendimiento al utilizar hardware especializado:

* **NumPy (CPU):** 1.06 segundos.
* **CuPy (GPU):** 156 milisegundos.
* **Speedup:** $\approx 6.8\times$.

**Conclusiones técnicas:**
1. **Paralelismo masivo:** La multiplicación de matrices se beneficia enormemente de los miles de núcleos CUDA de la GPU del clúster Bohr, ya que cada elemento de la matriz resultante puede calcularse de forma prácticamente independiente.
2. **Eficiencia en punto flotante:** Al utilizar `float32`, maximizamos la capacidad de cómputo de los núcleos de precisión simple de la GPU.
3. **Optimización de memoria:** Aunque mover matrices de $7000 \times 7000$ a la VRAM tiene un coste, el tiempo ahorrado en el cálculo compensa con creces la transferencia inicial.

Celda de PyTorch

In [5]:
import torch
import time

# Comprobar si CUDA está disponible (debería estarlo en Bohr)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

# Crear tensores directamente en la GPU
# A y B ya los tienes definidos de las celdas anteriores
A_torch = torch.from_numpy(A).to(device)
B_torch = torch.from_numpy(B).to(device)

# Warm-up
C_torch = torch.matmul(A_torch, B_torch)

# Medir tiempo con PyTorch
t0 = time.perf_counter()
C_torch = torch.matmul(A_torch, B_torch)
torch.cuda.synchronize() # Importante para medir tiempo real en GPU
t1 = time.perf_counter()

print(f"Tiempo PyTorch (GPU): {t1 - t0:.4f} s")

Usando dispositivo: cuda


/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GeForce GTX 1080 which is of cuda capability 6.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (7.0) - (12.0)
    
  warnings.warn(
/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:304: UserWarning: 
    Please install PyTorch with a following CUDA
    configurations:  12.6 following instructions at
    https://pytorch.org/get-started/locally/
    
  warnings.warn(matched_cuda_warn.format(matched_arches))
/usr/local/lib/python3.13/site-packages/torch/cuda/__init__.py:326: UserWarning: 
NVIDIA GeForce GTX 1080 with CUDA capability sm_61 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_70 sm_75 sm_80 sm_86 sm_90 sm_100 sm_120.
If you want to use the NVIDIA GeForce GTX 1080 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  wa

Tiempo PyTorch (GPU): 0.3286 s


### Nota sobre la ejecución de PyTorch
Se observa un `UserWarning` indicando que la arquitectura de la GPU (NVIDIA GeForce GTX 1080, capacidad 6.1) no es totalmente compatible con la versión de PyTorch instalada (que requiere capacidad 7.0 o superior). 

A pesar de la advertencia de compatibilidad, el código se ha ejecutado correctamente en la cola de **Bohr**, obteniendo un tiempo de **0.3286 s**, lo cual es significativamente más rápido que la versión de CPU, aunque ligeramente inferior al rendimiento de CuPy debido probablemente a que PyTorch no puede utilizar todos sus kernels optimizados para esta arquitectura antigua.